<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Lineare Regression

Der einfachste Fall der Lineraen Regression ist die Vorhersage einer (*abhängigen*) Variablen durch **eine** *unabhängige* Variable.
Dieser Spezialfall wird daher auch *einfache Lineare Regression* (ELR) oder *univariate Lineare Regression* genannt.

Wir wollen die ELR an einem einfachen Datensatz erproben.
Dazu generieren wir uns $N=30$ Datenpunkte im Bereich $x\in[0,10]$.
Die $y$-Werte sollen grob entlang einer geraden liegen.
Wir wählen für die gerade eine Steigung von $2$ und den Achsenabschnitt $-5$.

Natürlich sollen die Punkte nicht alle **auf** dieser Geraden liegen.
Daher fügen wir noch einen kleinen *Störfaktor* ein.
Wir addieren zu jedem $y$-Wert einen Wert aus dem Bereich $[-1,1]$.

In [ ]:
np.random.seed(42)
N = 30
x = 10 * np.random.rand(N)
y = 2 * x - 5 + 2*np.random.randn(N)
plt.scatter(x, y);
plt.show()

Wir können nun die *Scikit-Learn* Klasse `LinearRegression` verwenden, um ein ELR Modell für den Datensatz aufzustellen.
Mit der `fit`-Methode passen wir das Modell an den Datensatz an.

*Hinweis:* Die `fit`-MEthode erwartet ein 2-dimensionales Feld als ersten Parameter. Unser `x` ist aber ein eindimensionales Array. Wir ändern daher mit der `reshape`-Methode die Dimension von `x` vor dem Aufruf von `fit` um.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
x = x.reshape((-1,1))
model.fit(x, y)

print("Steigung:       ", model.coef_[0])
print("Achsenabschnitt:", model.intercept_)



Wir können nun die Modellfunktion plotten indem wir zunächst einige neue Datenpunkte erzeugen (`xplot`) und für diese Punkte den $y$-Wert über die `predict`-Methode schätzen.

Wir sehen, dass die Gerade die blauen Datenpukte unseres Trainingsdatensatzes sehr gut generalisiert.

In [ ]:
xplot = np.linspace(0, 10, 20).reshape((-1,1))
yplot = model.predict(xplot)

plt.scatter(x, y)
plt.plot(xplot, yplot, c='r')
plt.show()

Der ``LinearRegression`` Schätzer kann nicht nur auf univariate Probleme angewendet werden, sondern er funktioniert auch für mehrere unabhängige Variable.
Die Modellfunktion einer allgemeinen (*multivariaten*) lineren Regression lautet:
$$
\hat{y} = a_0 + a_1 x_1 + a_2 x_2 + \cdots + a_n x_n
$$

Dabei sind die $x_i$ die einzelnen unabhängigen Variablen, die Werte $a_i$ sind die zu trainierenden Modellparameter.
Der Parameter $a_0$, den wir beim univariaten Modell *Achsenabschnitt* genannt haben, heißt auch *Bias*-Parameter.
Es ist der einzige Parameter, der unabhängig von den Variablen ins Modell eingeht.

$\hat{y}$ ist der geschätzte Wert der Zielvariablen für den Vektor $\textbf{x}$.
Für unsere Trainingsdaten kennen wir den jeweiligen exakten Wert $y$.
Über einen Vergleich von $\hat{y}$ und $y$ kann man feststellen, wie *gut* die Schaätzung der Modellfunktion ist.

In [ ]:
np.random.seed(123)
N = 30
X = 10 * np.random.rand(N, 3)
noise = np.random.normal(0, 0.5, (N,3))
y = 0.5 + np.dot(X+noise, [1.5, -2., 1.]) + 0.5*np.random.randn(1)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,8))
axs[0].scatter(X[:,0], y)
axs[1].scatter(X[:,2], y)
axs[2].scatter(X[:,1], y)
axs[0].set_ylabel('y')
axs[0].set_xlabel(r'$x_0$')
axs[1].set_xlabel(r'$x_1$')
axs[2].set_xlabel(r'$x_2$')
plt.show()

**Aufgabe:** Trainieren Sie ein `LinearRegression` Modell mit dem Datensatz `X` und geben Sie die gelerneten Modellparameter $a_0$ bis $a_3$ aus.

In [ ]:
model = LinearRegression()
# YOUR CODE HERE
raise NotImplementedError()

### Preisvorhersage für Häuser in Boston

Im folgenden Beispiel verwenden wir einen sehr verbreitet verwendeten Datensatz für die Demonstration von Regressions-Aufgaben, den *Boston Housing Prices*. 
Die Daten wurden 1978 gesammelt und jeder der 506 Einträge enthält Informationen zu 14 Merkmalen von Häusern aus verschiedenen Vororten in Boston.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.datasets import load_boston

dataset = load_boston()
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df.head()

Der Datensatz enthält folgende Merkmale:
- Kriminalitätsrate pro Kopf (crim)
- Anteil des Wohnlandes über 25.000 sq.ft (zn)
- Anteil der Nicht-Handels-Geschäftsfläche (indus)
- Flusslage am Chas River (chas: 1=am Fluss, 0=nicht am Fluss)
- Konzentration der Stickstoffoxide (Teile pro 10 Millionen, nox)
- durchschnittliche Anzahl Zimmer pro Wohnung (rm)
- Anteil der Eigentumswohnungen, die vor 1940 gebaut wurden (age; der Datensatz wurde 1978 veröffentlicht)
- gewichtete Entfernung zu fünf Bostoner Beschäftigungszentren (dis)
- Index der Zugänglichkeit zu Einfallstraßen (rad)
- vollwertiger Immobilien-Steuersatz pro $10.000 (tax)
- Schüler-Lehrer-Quotient (ptratio)
- Anteil Farbiger: (black; Formel: 1000(Bk – 0.63)²)
- Bevölkerungsanteil mit niedrigem Status (lstat)

Die Zielvariable `target` ist der Preis der jeweiligen Häuser in Tausend US-Dollar

In [ ]:
preise = dataset.target
print(f"Der Wert der Häuser reicht von {int(np.min(preise)*1000)} bis {int(np.max(preise)*1000)} USD")
print(f"Im Mittel kosten die Häuser {int(np.mean(preise)*1000)} USD")

**Aufgabe:** In der folgenden Code-Zelle teilen wir den Datensatz auf, in einen Trainings- und in einen Testdatensatz.
Verwenden Sie die Trainingsdaten, um ein lineares Regressionsmodell zu trainieren.
Berechnen Sie damit eine Schätzung `y_vorhersage` für die Testdaten.
Danach berechnen wird den *mittleren absoluten Fehler* und überprüfen damit, wie gut unser Modell die Preise der Häuser schätzen kann.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.3, random_state=42)

y_vorhersage = None
# YOUR CODE HERE
raise NotImplementedError()

mittlerer_fehler = np.sum(np.abs(y_vorhersage-y_test))/len(y_vorhersage)
print(f"Der mittlere Fehler der Vorhersage liegt bei {int(mittlerer_fehler*1000)} USD")

## California Housing

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.datasets import fetch_california_housing

dataset = fetch_california_housing()
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df.head()


Der Datensatz enthält folgende Merkmale:
- Kriminalitätsrate pro Kopf (crim)
- Anteil des Wohnlandes über 25.000 sq.ft (zn)
- Anteil der Nicht-Handels-Geschäftsfläche (indus)
- Flusslage am Chas River (chas: 1=am Fluss, 0=nicht am Fluss)
- Konzentration der Stickstoffoxide (Teile pro 10 Millionen, nox)
- durchschnittliche Anzahl Zimmer pro Wohnung (rm)
- Anteil der Eigentumswohnungen, die vor 1940 gebaut wurden (age; der Datensatz wurde 1978 veröffentlicht)
- gewichtete Entfernung zu fünf Bostoner Beschäftigungszentren (dis)
- Index der Zugänglichkeit zu Einfallstraßen (rad)
- vollwertiger Immobilien-Steuersatz pro $10.000 (tax)
- Schüler-Lehrer-Quotient (ptratio)
- Anteil Farbiger: (black; Formel: 1000(Bk – 0.63)²)
- Bevölkerungsanteil mit niedrigem Status (lstat)

Die Zielvariable `target` ist der Preis der jeweiligen Häuser in Tausend US-Dollar

MedInc	HouseAge	AveRooms	AveBedrms	Population	AveOccup	Latitude	Longitude


Der Datensatz enthält folgende Merkmale:
- **MedInc:** Median des Einkommens im Bezirk
- **HouseAge:** Median des Hausalters im Bezirk
- **AveRooms:** durchschnittliche Anzahl der Zimmer pro Haushalt
- **AveBedrms:** Durchschnittliche Anzahl der Schlafzimmer pro Haushalt
- **Population:** Bezirkbevölkerung
- **AveOccup:** durchschnittliche Anzahl der Haushaltsmitglieder
- **Latitude:** Bezirk Breitengrad
- **Longitude:** Bezirk Längengrad

## Quellen:
[1] Jake VanderPlas, [*Python Data Science Handbook*](https://jakevdp.github.io/PythonDataScienceHandbook), O'Reilly, 2016.\
[2] Wolf Riepl, [*Machine Learning mit R und caret: GBM optimieren (Gradient Boosting Machine)*](https://statistik-dresden.de/archives/14967), Artikel auf https://statistik-dresden.de,  Veröffentlicht am 23.01.2018 (zugegriffen am 27.04.2021)